In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In the previous session, you were told which parameters to use for the prior normalinverse-gamma distribution. This time you have to choose the prior parameters yourself.
Given the information below, find reasonable values for the prior parameters of the
normal-inverse-gamma distribution — μ₀, ν₀, α₀, β₀. You will be asked to provide your
values for the prior hyperparameters in class, and to explain how you came up with them.

Information provided:
● The data are normally distributed. The error margins given below represent 1
standard deviation from the mean of the parameter.
● Constraint: the mean of the data is approximately 2.3 ± 0.5.
● Constraint: the variance of the data is approximately 2.75 ± 1.
● Find μ₀, ν₀, α₀, β₀ hyperparameters for the normal-inverse-gamma prior that
match this information.

You can solve this problem in at least two different ways. You have to implement at least
one of these two methods. Ideally, implement both and check that your answers match.
1. Frame the information above as an optimization problem. You should design a
function that is minimized when the constraints above are all satisfied. You can do
this by creating an objective function of the sum of squared residuals of the
constraints. Use SciPy to find the minimum of the objective function.
For example, the mean of the data mean should be 2.3. From Wikipedia , we know
that the expected value of the mean of the normal-inverse-gamma distribution is μ0
so we add the term (μ .3) to the objective function. You should have 4 such 0 ­ 2
2
terms in your function, corresponding to the 4 constraints in the information above.

In [9]:
'''
Function definitions for the normal-inverse-gamma distribution. The parameters
of the distribution, namely mu (μ), either lambda (λ) or nu (ν), alpha (α),
beta (β), are used as defined here:

  https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution

Note that we use the symbol nu (ν) rather than lambda (λ) for the third
parameter. This is to match the notation used in the conjugate priors table on
Wikipedia:

  https://en.wikipedia.org/wiki/Conjugate_prior#Table_of_conjugate_distributions
'''

def norminvgamma_pdf(x, sigma2, mu, nu, alpha, be):
    '''
    The probability density function of the normal-inverse-gamma distribution at
    x (mean) and sigma2 (variance).
    '''
    return (
        sts.norm.pdf(x, loc=mu, scale=np.sqrt(sigma2 / nu)) *
        sts.invgamma.pdf(sigma2, a=alpha, scale=be))

def norminvgamma_rvs(mu, nu, alpha, be, size=1):
    '''
    Generate n samples from the normal-inverse-gamma distribution. This function
    returns a (size x 2) matrix where each row contains a sample, (x, sigma2).
    '''
    # Sample sigma^2 from the inverse-gamma distribution
    sigma2 = sts.invgamma.rvs(a=alpha, scale=be, size=size)
    # Sample x from the normal distribution
    x = sts.norm.rvs(loc=mu, scale=np.sqrt(sigma2 / nu), size=size)
    return np.vstack((x, sigma2)).transpose()

In [6]:
import numpy as np
from scipy.optimize import minimize

### Define the objective function 
##define the hyperparameters using the equations from wikepedia
def func(x):
    mu, nu, alpha, beta = x[0],x[1],x[2],x[3]
    x= mu
    var_x=  (beta / (alpha -1) *nu)
    sigma2 = (beta / (alpha -1))
    var_sigma2 = (beta**2)  / (((alpha -1)**2) * (alpha -2))
    return (x - 2.3)**2 + (var_x- 0.5)**2+ (sigma2 -2.75)**2 + (var_sigma2 - 1)**2 

for x_initial in [
np.array([0, 0.054, 2.2, 0.4]),
np.array([2, 0.006, 2.4, 0.5]),
np.array([1, 0.009, 2.1, 0.7]),
np.array([10, 10, 20, 15])
]:
    result = minimize(func, x_initial)
    x_final = result.x
    print('Started at', x_initial)
    print('Ended at', x_final)
    print('func(%s) = %.2f' % (x_final, func(x_final)))
    print()

Started at [ 0.     0.054  2.2    0.4  ]
Ended at [  2.3          0.18181832   9.5624952   23.54685931]
func([  2.3          0.18181832   9.5624952   23.54685931]) = 0.00

Started at [ 2.     0.006  2.4    0.5  ]
Ended at [  2.29999961   0.18181839   9.5624978   23.54686811]
func([  2.29999961   0.18181839   9.5624978   23.54686811]) = 0.00

Started at [ 1.     0.009  2.1    0.7  ]
Ended at [  2.30000001   0.18181805   9.5625037   23.54689073]
func([  2.30000001   0.18181805   9.5625037   23.54689073]) = 0.00

Started at [10 10 20 15]
Ended at [  2.29999991   0.18181824   9.56250047  23.54687648]
func([  2.29999991   0.18181824   9.56250047  23.54687648]) = 0.00



2. The constraints above give you the mean of the data mean, the standard deviation
of the data mean, the mean of the data variance, and the standard deviation of the
data variance. Solve these 4 equations (one for each of the quantities above)
simultaneously to find the values of the prior parameters.

In [9]:
#Solving the four equations simultaneously using a numerical solver 
# 2.3= mu
# 0.5=  (beta / (alpha -1) *nu)
# 2.75 = (beta / (alpha -1))
# 1 = (beta**2)  / (((alpha -1)**2) * (alpha -2))
